In [40]:
import robin_stocks as r 
from datetime import datetime
from dateutil import tz
import pandas as pd
import plotly.express as px

from_zone = tz.tzutc()
to_zone = tz.tzlocal()
def utcToLocal(strDate):
    utc = datetime.strptime(strDate, '%Y-%m-%dT%H:%M:%SZ')
    utc = utc.replace(tzinfo=from_zone)
    return utc.astimezone(to_zone)

import configparser
config = configparser.RawConfigParser()
configFilePath = '/Users/philipmassey/.tokens/robinhood.cfg'
config.read(configFilePath)
rhuser = config.get('login', 'user')
rhpwd = config.get('login', 'pwd')
login = r.login(rhuser,rhpwd)

def earningsForCurrentYear(symbols):
    this_year = '2020'
    previous_month = '04'
    current_year_dates = []
    for symbol in symbols:
        try:
            earnings = r.get_earnings(symbol=symbol)
            if len(earnings) > 0:
                current_year = list(filter(lambda e:e['report']['date'][0:4]== this_year, earnings))
                earnings_dates = list(map(lambda e:e['report']['date'], current_year))
                earnings_dates = list(filter(lambda e:e[5:7]>previous_month,earnings_dates))
                current_year_dates.append((symbol,earnings_dates))
            else:
                print(symbol,'no earnings')
        except Exception as e:
            print('skipped',symbol,e)

    return sorted(current_year_dates,key=lambda x: x[1][0])


In [42]:
df_options = pd.DataFrame(r.options.get_open_option_positions())
myoptions = df_options['chain_symbol']
current_year_earnings = earningsForCurrentYear(myoptions)
for e in current_year_dates:
    print(e[0],'\t',e[1])
     

BLDP 	 ['2020-05-05', '2020-07-29']
SPG 	 ['2020-05-11', '2020-07-30']
HD 	 ['2020-05-19', '2020-08-18']
LOW 	 ['2020-05-20', '2020-08-19']
OKTA 	 ['2020-05-28', '2020-08-26']


In [34]:
df_holdings = pd.DataFrame(r.build_holdings())
df

,account,account_number,average_buy_price,created_at,instrument,intraday_average_buy_price,intraday_quantity,pending_average_buy_price,quantity,shares_held_for_buys,shares_held_for_options_collateral,shares_held_for_options_events,shares_held_for_sells,shares_held_for_stock_grants,shares_pending_from_options_events,updated_at,url
0,https://api.robinhood.com/accounts/5QU67573/,5QU67573,21.0250,2020-04-28T15:25:04.671202Z,https://api.robinhood.com/instruments/2b9c860d...,0.0000,0.00000000,21.0250,50.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,2020-04-28T15:25:05.282508Z,https://api.robinhood.com/positions/5QU67573/2...
1,https://api.robinhood.com/accounts/5QU67573/,5QU67573,2374.1126,2020-05-14T22:30:51.301699Z,https://api.robinhood.com/instruments/c0bb3aec...,2374.1126,0.04212100,2374.1126,0.04212100,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,2020-05-15T13:35:32.667364Z,https://api.robinhood.com/positions/5QU67573/c...
